<a href="https://colab.research.google.com/github/shinjangwoon/jangwoonshin/blob/practice/Magic_Formula.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd

csv_url = '/content/drive/MyDrive/Colab Notebooks/company.csv'

company = pd.read_csv(csv_url)
print(company.shape)
company.head(10)

In [ ]:
type(company)

In [ ]:
type(company['종목코드'])

In [ ]:
company[['회사명', '종목코드']].head(10)

In [ ]:
#company.loc[100:110]
ids = [1, 5, 100, 2010]
column_names = ['분류', '회사명', '종목코드']
company.loc[ids][column_names]

In [ ]:
# refactoring
company.loc[ids,column_names]

In [ ]:
# true 값만 보이게 함
kospi = company[company['분류'] == '코스피']
kosdaq = company[company['분류'] == '코스닥']

In [ ]:
ids = [1, 4, 7, 10]
kospi.loc[ids, column_names]

In [ ]:
company[company['회사명'] == '카카오'][column_names]

In [ ]:
# 관심있는 회사 목록 list
interested = ['한화', '삼성전자', '카카오']

#list를 사용한 복수의 기업 DataFrame filtering(찾고자 하는 이름이 정확히 일치해야함)
company[company['회사명'].isin(interested)]

### Missing value == 결측치

In [ ]:
# 결측치 확인
# NaN == Not a Number
company[company['종목코드'].isnull()]

In [ ]:
# notnull == null 값이 아닌 것들만 찾아줌
company[company['종목코드'].notnull()]

In [ ]:
# 결측치 값 삭제
# NaN 값이였던 14번 항목이 날라간 것을 확인가능
company['종목코드'].dropna()[:15]

In [ ]:
# 하나라도 NaN 값이면 날려버림
#company.dropna(how='any')

# 전부가 NaN 값이면 날려버림
#company.dropna(how='all')

#company.dropna(inplace=True)
company = company.dropna()

In [ ]:
#(REITs) Real Estate Investment Trust Corps.
#(SPAC) Special Purpose Acquisition Corps.

In [ ]:
# 문자열 안에서 ' '이라는 문자가 들어있는 모든 것들을 찾아줌
# NaN값을 없애고 실행해야 함
#company[company['회사명'].str.contains('삼성')]



# ~을 추가하면 반대로 문자가 들어있는 것을 제외하고 찾아줌
#company[~company['회사명'].str.contains('삼성')]

#company[company['회사명'].str.contains('스팩')]


In [ ]:
# 여러 개의 단어를 없애고 싶을때  | == or
company = company[~company['회사명'].str.contains('스팩|리츠')]


In [ ]:
company

In [ ]:
'''
업종 - 기타 금융업 
코스닥에 상장
2015-1-1 이후에 상장된 기업 정보만 필터링 - datetime 이나 pandas의 Timestamp 메소드 방법 사용
read_csv의 parameter 중 parse_dates 사용 - pandas read csv parse dates
to_datetime 함수 사용 - pandas to datetime

timestamp 사용
'''

### 첫번째 방법

In [ ]:
company = pd.read_csv(csv_url, parse_dates=['상장일'])
company['상장일']

In [ ]:
financial_corps = company[company['업종'] == '기타 금융업']
financial_corps_kosdaq = financial_corps[financial_corps['분류'] == '코스닥']


In [ ]:
# python -> string
# pandas -> object
financial_corps_kosdaq[financial_corps_kosdaq['상장일']  > pd.Timestamp(2015, 1, 1)]

# Magic Formula(마법 공식)

## Basic setting

In [ ]:
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
#import urllib.request
from urllib import request as rq

In [ ]:
FS_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A%s&cID=&MenuYn=Y&ReportGB=&NewMenuID=103&stkGb=701'
INDEX_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Invest.asp?pGB=1&gicode=A%s&cID=&MenuYn=Y&ReportGB=&NewMenuID=105&stkGb=701'

sample_code = '005930'

## Get EV/EBITDA
### Earnings before Interest, Tax, Depreciation, Amortization

In [ ]:
index_html = rq.urlopen(INDEX_URL%sample_code).read()

index_soup = BeautifulSoup(index_html, 'html.parser')


In [ ]:
ev_cells = index_soup.find('tr', {'id':'p_grid1_14'}).find_all('td')
ev = float(ev_cells[3].string.replace(',',''))

##Get Gross Profit

In [ ]:
fs_html = rq.urlopen(FS_URL%sample_code).read()
fs_soup = BeautifulSoup(fs_html, 'html.parser')

In [ ]:
gp_cells = fs_soup.find('div', {'id':'divSonikY'}).find_all('tr', {'class': "rwf"})
gross_profit = float(gp_cells[2].find_all('td')[2].string.replace(',',''))
gross_profit

##Get Asset

In [ ]:
asset_cells = fs_soup.find('div', {'id' : 'divDaechaY'}).find_all('tr', {'class': "rwf"} )
asset = float(asset_cells[0].find_all('td')[2].string.replace(',',''))

##Get All Stock code

In [ ]:
csv_url = '/content/drive/MyDrive/Colab Notebooks/company.csv'

company = pd.read_csv(csv_url)
code_list = company['종목코드']
sample_code_list = code_list[1000:1050]
sample_code_list


## Create Crawling Function

In [ ]:
def crawler(li):
    FS_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A%s&cID=&MenuYn=Y&ReportGB=&NewMenuID=103&stkGb=701'
    INDEX_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Invest.asp?pGB=1&gicode=A%s&cID=&MenuYn=Y&ReportGB=&NewMenuID=105&stkGb=701'

    result = {}

    for code in tqdm(li):
        try:
            # get company name 
            name = company[company['종목코드'] == code]['회사명'].values[0]

            # get EV/EBITDA
            index_html = rq.urlopen(INDEX_URL%code).read()

            index_soup = BeautifulSoup(index_html, 'html.parser')
            ev_cells = index_soup.find('tr', {'id':'p_grid1_14'}).find_all('td')
            ev = float(ev_cells[3].string.replace(',',''))
            fs_html = rq.urlopen(FS_URL%sample_code).read()
            fs_soup = BeautifulSoup(fs_html, 'html.parser')

            # get Gross profit
            fs_html = rq.urlopen(FS_URL%code).read()
            fs_soup = BeautifulSoup(fs_html, 'html.parser')
            gp_cells = fs_soup.find('div', {'id':'divSonikY'}).find_all('tr', {'class': "rwf"})
            gross_profit = float(gp_cells[2].find_all('td')[2].string.replace(',',''))

            # get Asset
            asset_cells = fs_soup.find('div', {'id' : 'divDaechaY'}).find_all('tr', {'class': "rwf"} )
            asset = float(asset_cells[0].find_all('td')[2].string.replace(',',''))

            # Insert into result dict
            result[name] = [code, ev, gross_profit, asset]

        except(TypeError, AttributeError, IndexError, ValueError) :
            pass

    # convert dict into DataFrame
    result_df = pd.DataFrame(result)

    # transposing DataFrame
    result_df = result_df.transpose()

    # set Column name
    result_df.columns = ['Code', 'EV_EBITDA', 'Gross_profit', 'Asset']

    return result_df

In [ ]:
result_df = crawler(sample_code_list)
tmp_df = result_df.copy()
tmp_df.head(10)

In [ ]:
tmp_df['GP_A'] = tmp_df['Gross_profit'] / tmp_df['Asset']
tmp_df

## Granding Scores
### 1.EV/EBITDA Score

In [ ]:
top_range =  int(len(tmp_df) * 0.3)


middle_range =  int(len(tmp_df) * 0.7)
print(middle_range, top_range)

In [ ]:
# 상위 30% 값인 데이터에서 코드를 뽑아 원본 데이터에 매칭을 시켜 있으면 True 아니면 False
tmp_df['Code'].isin(tmp_df.sort_values(by='EV_EBITDA')[:top_range]['Code'])

In [ ]:

tmp_df.loc[tmp_df['Code'].isin(tmp_df.sort_values(by='EV_EBITDA')[:top_range]['Code']),'EV_EBITDA_Score'] = 3

tmp_df.loc[tmp_df['Code'].isin(tmp_df.sort_values(by='EV_EBITDA')[top_range:middle_range]['Code']),'EV_EBITDA_Score'] = 2

tmp_df.loc[tmp_df['Code'].isin(tmp_df.sort_values(by='EV_EBITDA')[middle_range:]['Code']),'EV_EBITDA_Score'] = 1


In [ ]:
tmp_df

### 2. GP/A Score

In [ ]:
# ascending=False == 내림차순
tmp_df.loc[ tmp_df['Code'].isin(tmp_df.sort_values(by='GP_A', ascending=False)[:top_range]['Code']) ,'GP_A_Score'] = 3

tmp_df.loc[ tmp_df['Code'].isin(tmp_df.sort_values(by='GP_A', ascending=False)[top_range:middle_range]['Code']) ,'GP_A_Score'] = 2

tmp_df.loc[ tmp_df['Code'].isin(tmp_df.sort_values(by='GP_A', ascending=False)[middle_range:]['Code']) ,'GP_A_Score'] = 1



In [ ]:
tmp_df

### Total Score

In [ ]:
tmp_df['Total_Score'] = tmp_df['EV_EBITDA_Score'] + tmp_df['GP_A_Score']
#tmp_df

In [ ]:
# Total_Score가 6인 것들만 뽑아 다른 dataframe에 넣기
max_point_df = tmp_df[tmp_df['Total_Score'] == 6.0]
max_point_df

In [ ]:
max_point_df.sort_values(by='GP_A', ascending=False)